In [16]:
!pip install anthropic

# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

## If you'd prefer to work in IDEs

If you're more comfortable in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  
If you'd prefer to work in VSCode, [here](https://chatgpt.com/share/676f2e19-c228-8012-9911-6ca42f8ed766) are instructions from an AI friend on how to configure it for the course.

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [3]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

import anthropic
from anthropic import Anthropic

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
#api_key = os.getenv('OPENAI_API_KEY')
api_key = os.getenv('CLAUDE_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    if api_key.startswith("sk-ant-"):
        print("An API key was found!  Looks like a CLAUDE key.  Looks good so far!")
    else:
        print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


An API key was found!  Looks like a CLAUDE key.  Looks good so far!


In [5]:
#openai = OpenAI()
anthropic_client = Anthropic(
    # Replace with your API key
    api_key=api_key
)
# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [11]:
prompt = "return results formatted with markdown. creationists beleive the earth is only 6000 years old.  How would a creationist explain the time it takes for light to reach earth from more than 6000 light years?"
message = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        messages=[{
            "role": "user",
            "content": prompt
        }]
    )
display(Markdown(message.content[0].text))

Some creationists attempt to explain the apparent age of the universe by suggesting that God created the universe with the appearance of age, including light en route from distant stars and galaxies. Here is a possible creationist explanation, formatted with Markdown:

---

### Creationist Explanation

Creationists who believe the Earth and universe are only around 6,000 years old have proposed various explanations for the observed phenomenon of light reaching Earth from objects more than 6,000 light-years away. One prominent explanation is the idea of "cosmic creation," which suggests that God created the universe with the appearance of age, including the creation of light beams already en route from distant stars and galaxies.

According to this view, when God created the universe, He did not create it as a mere seed or a point of origin. Instead, He created the entire universe in a fully-formed state, complete with stars, galaxies, and the light that had apparently been traveling for billions of years already in motion.

Proponents of this idea argue that since God is omnipotent and not bound by natural laws, He could have created the universe in a mature state, including the illusion of age, without violating any theological principles. They claim that this explanation resolves the apparent contradiction between the young age of the universe and the observation of distant starlight, which would require billions of years to reach Earth under natural processes.

Critics of this explanation often argue that it appears to be an ad hoc hypothesis designed solely to reconcile scientific observations with a literal interpretation of the Genesis account. They suggest that accepting such an explanation would imply that God intentionally created the universe with the appearance of deception, which some consider incompatible with the nature of an omniscient and truthful deity.

---

It's important to note that this explanation is not universally accepted within the creationist community, and there are ongoing debates and diverse viewpoints on this subject among creationists and within the broader scientific and theological communities.

## OK onwards with our first project

In [12]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [13]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [14]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [15]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [16]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [17]:
messages = [
    {"role": "user", "content": "return contents in markdown"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [18]:
#prompt = "return results formatted with markdown. creationists beleive the earth is only 6000 years old.  How would a creationist explain the time it takes for light to reach earth from more than 6000 light years?"
message = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        messages=messages
    )
display(Markdown(message.content[0].text))

```
4
```

## And now let's build useful messages for GPT-4o-mini, using a function

In [27]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [28]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder 

## Time to bring it together - the API for OpenAI is very simple!

In [33]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = anthropic_client.messages.create(
        model = "claude-3-sonnet-20240229",
        max_tokens=1000,
        messages = messages_for(website)
    )
    return response.content[0].text

In [34]:
summarize("https://edwarddonner.com")

'Here\'s a summary of the website in markdown format:\n\n## Home - Edward Donner\n\nThis website appears to be a personal blog and portfolio site for Edward Donner, who is the co-founder and CTO of Nebula.io, a company that applies AI to talent discovery and recruitment. The website features the following sections:\n\n### Home\n\nThe homepage introduces Edward Donner and provides a brief overview of his interests, including coding, experimenting with LLMs, DJing, and electronic music production. It also mentions his previous startup, untapt, which was acquired in 2021.\n\n### Outsmart\n\nThis section describes an arena called "Outsmart," where LLMs (Large Language Models) compete against each other in a battle of diplomacy and deviousness.\n\n### About\n\nThis section likely provides more detailed information about Edward Donner and his background.\n\n### Posts\n\nThe website includes a blog section with the following recent posts:\n\n1. **Welcome, SuperDataScientists!** (December 21, 

In [35]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [36]:
display_summary("https://edwarddonner.com")

Here's a summary of the website "Home - Edward Donner" in Markdown format:

# Edward Donner's Website

## Overview
This website belongs to Edward Donner, who is the co-founder and CTO of Nebula.io, a company that applies AI to help people discover their potential and pursue their reason for being. The website features information about Edward, his interests, and his work with AI and language models (LLMs).

## Announcements/Posts

### Outsmart: An LLM Arena
The website promotes "Outsmart," an arena that pits LLMs against each other in a battle of diplomacy and deviousness.

### Recent Posts
1. **Welcome, SuperDataScientists!** (December 21, 2024)
2. **Mastering AI and LLM Engineering – Resources** (November 13, 2024)
3. **From Software Engineer to AI Data Scientist – Resources** (October 16, 2024)
4. **Outsmart LLM Arena – A Battle of Diplomacy and Deviousness** (August 6, 2024)

## About Edward Donner
Edward Donner is interested in writing code, experimenting with LLMs, DJing, amateur electronic music production, and reading Hacker News. He is the co-founder and CTO of Nebula.io, a company that applies AI to help people discover their potential and pursue their reason for being. Previously, he founded and was the CEO of the AI startup untapt, which was acquired in 2021.

## Connect
The website provides various ways to connect with Edward Donner, including his email address, LinkedIn, Twitter, Facebook, and a newsletter subscription form.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [37]:
display_summary("https://www.youtube.com/@Simon-fk5su")

Here's a short summary of the website titled "Simon (Walking in L.A.) - YouTube" in markdown:

## YouTube Website Summary

This website appears to be a page on YouTube, a popular video-sharing platform owned by Google. The contents listed seem to be standard links and sections found on the YouTube website, including:

- **About**: Information about YouTube and its services.
- **Press**: Resources and information for members of the media.
- **Copyright**: Guidelines and policies related to copyright and intellectual property on YouTube.
- **Contact us**: Options to get in touch with YouTube for various inquiries or issues.
- **Creators**: Resources and tools for content creators on the platform.
- **Advertise**: Information and options for advertising on YouTube.
- **Developers**: Resources and tools for developers who wish to integrate with YouTube's APIs and services.
- **Terms**: YouTube's terms of service and legal agreements.
- **Privacy**: YouTube's privacy policies and information about data collection and usage.
- **Policy & Safety**: Guidelines and policies related to content moderation, safety, and community standards on YouTube.
- **How YouTube works**: Explanations and information about YouTube's platform and features.
- **Test new features**: Options for users to test and provide feedback on new features or updates.
- **NFL Sunday Ticket**: Information about YouTube's partnership with NFL Sunday Ticket for live streaming NFL games.
- **© 2025 Google LLC**: Copyright information indicating that YouTube is owned and operated by Google LLC.

There doesn't seem to be any specific news or announcements mentioned in the provided contents.

In [31]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is a company focused on AI safety and research, headquartered in San Francisco. The site emphasizes their flagship AI models, particularly Claude 3.5 Sonnet, which is highlighted as the most advanced version of their intelligent AI models.

## Key Features:
- **Claude 3.5 Sonnet**: The latest intelligent AI model from Anthropic, now available for users.
- **API & Enterprise Solutions**: There are options to build custom applications and enterprise solutions using Claude.
- **AI Safety Research**: The company prioritizes safety in AI development, with a multidisciplinary team working on creating beneficial AI technologies.

## Recent Announcements:
1. **New AI Models**: 
   - On **October 22, 2024**, Anthropic announced the introduction of new models, including Claude 3.5 Sonnet and Claude 3.5 Haiku.
2. **Claude for Enterprise**: 
   - Announced on **September 4, 2024**, this caters to businesses leveraging AI technology.
3. **Research Initiatives**: 
   - Notable research contributions include discussions on **Constitutional AI** focused on harmlessness in AI (December 15, 2022) and **Core Views on AI Safety** (March 8, 2023).

The website also features sections dedicated to the team, career opportunities, and a comprehensive overview of their products and research initiatives.

In [32]:
display_summary("https://www.linkedin.com/in/simon-kaltgrad/")

# Summary of Simon Kaltgrad's LinkedIn Profile

**Profile Overview**  
Simon Kaltgrad is a seasoned IT leader based in Las Vegas, Nevada, with over 25 years of experience, particularly in software and application development. Currently affiliated with UrVenue, he emphasizes the importance of effective team collaboration and Agile methodologies in software development.

**Key Contributions**  
Simon has shared various insights through articles focusing on:
1. **Application Development Team Focus**: Emphasizes setting tangible goals for team members to foster satisfaction and productivity.
2. **Effective Agile Management**: Advocates tailored best practices for Agile methodologies based on team needs and stresses responsibility among team members.
3. **Resilience in Software Development**: Highlights the importance of keeping a big-picture perspective under time constraints and dealing with errors constructively.
4. **Addressing Skill Gaps**: Views skill gaps as opportunities for delegation and team training.
5. **Delegating Effectively**: Underlines the significance of trust in the delegation process to enhance job satisfaction.

**Recent Engagements**  
- Liked the announcement of the opening of Toro by Chef Richard Sandoval at Fairmont Chicago.
- Engaged with job alerts for talent acquisition roles in Canada's Western Mountain Region.
- Celebrated festive moments with colleagues and shared updates on community events, showcasing his involvement in team culture and hospitality.

Simon's profile reflects a commitment to advancing in IT leadership, fostering agile environments, and nurturing team dynamics to achieve collective success.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293